In [7]:
import os
import cv2
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import joblib

dataset_path = r"C:\Users\lsj24\OneDrive\Desktop\pc_diagnose/image"
categories = ['autumnwarm', 'springwarm', 'summercool', 'wintercool']

def extract_feature(img_path):
    try:
        img = cv2.imread(img_path)
        img = cv2.resize(img, (100, 100))
        img_lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
        b_avg = np.mean(img_lab[:, :, 2])  # LAB의 B 채널 평균
        return [b_avg]
    except:
        return None

X = []
y = []

for label in categories:
    folder = os.path.join(dataset_path, label)
    if not os.path.exists(folder):
        print(f"{folder} 경로가 존재하지 않습니다.")
        continue
    for file in os.listdir(folder):
        img_path = os.path.join(folder, file)
        feature = extract_feature(img_path)
        if feature:
            X.append(feature)
            y.append(label)

print("샘플 개수:", len(X))

le = LabelEncoder()
y_encoded = le.fit_transform(y)

print("레이블 클래스:", le.classes_)

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

model = RandomForestClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("정확도:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=le.classes_))

# 모델 저장
joblib.dump(model, 'sklearn_personal_color_model.pkl')
joblib.dump(le, 'label_encoder.pkl')


샘플 개수: 213
레이블 클래스: ['autumnwarm' 'springwarm' 'summercool' 'wintercool']
정확도: 0.37209302325581395
              precision    recall  f1-score   support

  autumnwarm       0.22      0.29      0.25         7
  springwarm       0.33      0.31      0.32        13
  summercool       0.46      0.75      0.57         8
  wintercool       0.44      0.27      0.33        15

    accuracy                           0.37        43
   macro avg       0.37      0.40      0.37        43
weighted avg       0.38      0.37      0.36        43



['label_encoder.pkl']

In [9]:
import joblib
import cv2
import numpy as np

# 저장한 모델과 라벨 인코더 불러오기
model = joblib.load('sklearn_personal_color_model.pkl')
le = joblib.load('label_encoder.pkl')

def extract_feature(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (100, 100))
    img_lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    b_avg = np.mean(img_lab[:, :, 2])
    return [b_avg]

# 예측할 이미지 경로 설정
test_image_path = r'C:\Users\lsj24\OneDrive\Desktop\pc_diagnose\test_image.jpg'  # 예시

feature = extract_feature(test_image_path)
predicted_label_num = model.predict([feature])[0]
predicted_label = le.inverse_transform([predicted_label_num])[0]

print(f"예측된 퍼스널 컬러: {predicted_label}")


예측된 퍼스널 컬러: autumnwarm


In [12]:
from PIL import Image
import numpy as np
import os
import cv2

dataset_test_path = r"C:\Users\lsj24\OneDrive\Desktop\pc_diagnose\test_dataset"
categories = ['autumnwarm', 'springwarm', 'summercool', 'wintercool']
img_size = 100
valid_extensions = ('.jpg', '.jpeg', '.png', '.bmp')

X_test = []
y_test = []

for label in categories:
    folder = os.path.join(dataset_test_path, label)
    if not os.path.exists(folder):
        print(f"{folder} 경로가 존재하지 않습니다.")
        continue
    for file in os.listdir(folder):
        if not file.lower().endswith(valid_extensions):
            continue
        img_path = os.path.join(folder, file)
        try:
            img_pil = Image.open(img_path).convert('RGB')
        except Exception as e:
            print(f"이미지 로드 실패(PIL): {img_path} / 에러: {e}")
            continue
        img = np.array(img_pil)
        img = cv2.resize(img, (img_size, img_size))
        img_lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
        b_avg = img_lab[:, :, 2].mean()
        X_test.append([b_avg])
        y_test.append(label)

print(f"총 테스트 이미지 개수: {len(X_test)}")
print(f"레이블 클래스: {set(y_test)}")


총 테스트 이미지 개수: 17
레이블 클래스: {'autumnwarm', 'summercool', 'wintercool', 'springwarm'}


In [13]:
import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

# 저장된 모델과 라벨 인코더 불러오기
model = joblib.load('sklearn_personal_color_model.pkl')
le = joblib.load('label_encoder.pkl')

# 테스트용 feature와 라벨
# 위에서 만든 X_test, y_test 사용 (y_test는 라벨 문자열 리스트)

# 라벨 문자열을 숫자 라벨로 변환
y_test_encoded = le.transform(y_test)

# 모델 예측
y_pred = model.predict(X_test)

# 정확도와 리포트 출력
accuracy = accuracy_score(y_test_encoded, y_pred)
print(f"테스트 데이터 정확도: {accuracy:.4f}")

print("\n분류 리포트:")
print(classification_report(y_test_encoded, y_pred, target_names=le.classes_))


테스트 데이터 정확도: 0.7647

분류 리포트:
              precision    recall  f1-score   support

  autumnwarm       0.67      0.50      0.57         4
  springwarm       1.00      0.75      0.86         4
  summercool       0.67      0.80      0.73         5
  wintercool       0.80      1.00      0.89         4

    accuracy                           0.76        17
   macro avg       0.78      0.76      0.76        17
weighted avg       0.78      0.76      0.76        17

